# Nivel 1

### Importación de la base de datos a Power BI a través de un script de Python 

In [ ]:
#Importamos las librerías necesarias para manejar los datos y crear las gráficas:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Importamos las librerías necesarias para la conexión con la base de datos:

import mysql.connector
from mysql.connector import Error

from sqlalchemy import create_engine

#Conectamos con la base de datos MySQL y cargamos las tablas:

#Credenciales de la base de datos:
usuario = 'root'
clave = 'root'
host = 'localhost'
db = 'transactions_alex'

#Importamos los datos de nuestra base de datos:
try:
    db_transactions = mysql.connector.connect(host = host,
                                         database = db,
                                         user = usuario,
                                         password = clave)
    
    #Nos aseguramos de que se realiza la conexión con MySQL Server y con la base de datos:
    if db_transactions.is_connected():
        db_Info = db_transactions.get_server_info()
        print(f"Conectado con la version {db_Info} de MySQL Server")
        cursor = db_transactions.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print(f"Conexión establecida con la base de datos {record[0]}")

    #Recuperamos el nombre de las tablas de la base de datos y los guardamos en una lista:
    query_tablas = """SHOW tables"""
    cursor.execute(query_tablas)
    db_tablas = cursor.fetchall()
    
    lista_tablas = []
    for tabla in db_tablas:
        lista_tablas.append(tabla[0])

    #Vamos a cargar la información de cada tabla en sendos dataframes y crearemos un diccionario
    #cuyas claves serán los nombres de las tablas almacenadas en la lista y sus valores, el dataframe correspondiente a cada tabla:
    conexion = create_engine(f'mysql+mysqlconnector://{usuario}:{clave}@{host}/{db}')
    dataframe = {}

    for tabla in lista_tablas:
        query = f"SELECT * FROM {tabla}"
        dataframe[tabla] = pd.read_sql(query, conexion)

    #Comprobamos que hemos cargado correctamente el diccionario
    for tabla in lista_tablas:
        print(f'Tabla {tabla}: \n', dataframe[tabla].head(), '\n')

#Manejamos los posibles errores de sesión
except Error as e:
    print("Error en la conexión a MySQL:", e)
    
#Cerramos el cursor y la conexión    
finally:
    if db_transactions.is_connected():
        cursor.close()
        db_transactions.close()
        print("Conexión a MySQL cerrada")

#Incorporamos los dataframes del script S801 a Power BI (y aprovechamos para limpiar el campo amount de la tabla product)

df_product = dataframe['product']
df_product['price'] = df_product['price'].str.replace('$','')
df_product['price'] = df_product['price'].astype(float)

#Nivel 1
#Ejercicio 1
df_transaction = dataframe['transaction']
df_transaction_ok = df_transaction[df_transaction['declined'] == 0]

#Ejercicio 2
df_usuario = dataframe['user']
df_usuario['birth_date'] = pd.to_datetime(df_usuario['birth_date'])
df_usuario['age'] = (pd.Timestamp('now') - df_usuario['birth_date']).dt.days // 365
df_usuario = df_usuario.rename(columns={'id': 'user_id'})
df_transaction_user = pd.merge(df_transaction, df_usuario, on='user_id', how='inner')

#Ejercicio 3
df_company = dataframe['company']

#Ejercicio 4
df_transaction['period'] = df_transaction['timestamp'].dt.to_period('M')
transacciones_mensuales = df_transaction['amount'].groupby([df_transaction['period']]).sum()
ventas_mensuales = df_transaction_ok['amount'].groupby([df_transaction['period']]).sum()

#Ejercicio 5
df_transaction_declined = df_transaction[df_transaction['declined']==1].groupby('period', as_index=False).size()

#Ejercicio 6
df_company.rename(columns={'id': 'company_id'}, inplace=True)
df_transaction_country = pd.merge(df_transaction, df_company, on='company_id', how='inner')
df_countries_agrup = df_transaction_country.groupby('country', as_index=False).size().sort_values('size', ascending=False)
lista_countries_top = df_countries_agrup[df_countries_agrup['size'] > 20]['country']
df_transaction_country_top = df_transaction_country[df_transaction_country['country'].isin(lista_countries_top)]

#Ejercicio 7
df_tr_pr = dataframe['transaction_products']
df_tr_pr.rename(columns={'transaction_id': 'id'}, inplace=True)
df_products_by_transaction = df_tr_pr.groupby('id', as_index=False).count()
df_products_by_transaction.rename(columns={'product_id':'prods_x_trans'}, inplace=True)
df_transaction_user_products = pd.merge(df_transaction_user, df_products_by_transaction, on='id', how='inner')

#Nivel 2
#Ejercicio 1
df_product.rename(columns={'id': 'product_id'}, inplace=True)
df_tr_pr_weight = pd.merge(df_tr_pr, df_product, on='product_id', how='left')
group_df_tr_pr_weight = df_tr_pr_weight.groupby('id')
df_tr_pr_weight_aggr = df_tr_pr_weight.pivot_table(index='id', values=['price', 'weight'], aggfunc=['sum'])
df_transaction_user_pr_wh = pd.merge(df_transaction_user_products, df_tr_pr_weight_aggr['sum'], on='id', how='left')
df_transaction_user_pr_wh_subset = df_transaction_user_pr_wh[['amount', 'age', 'prods_x_trans', 'price', 'weight']]

#Nivel 3
#Ejercicio 2
df_transaction_user_pr_wh['period'] = df_transaction_user_pr_wh['timestamp'].dt.to_period('M')
df_transaction_user_pr_wh['month'] = df_transaction_user_pr_wh['timestamp'].dt.month
df_transaction_user_pr_wh['year'] = df_transaction_user_pr_wh['timestamp'].dt.year

### Ejercicio 1

Gráfico con una variable numérica

In [ ]:
# El codi següent per crear un marc de dades i suprimir files duplicades sempre s'executa i funciona com a preàmbul de l'script: 

# dataset = pandas.DataFrame(amount)
# dataset = dataset.drop_duplicates()

# Enganxeu o escriviu el codi de l'script aquí:

df_transaction_ok = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Vamos a sacar algunas estadísticas sobre los ingresos y a graficarlas:
amount = df_transaction_ok['amount']

media = amount.mean()
mediana = amount.median()
percentil_25 = np.percentile(amount,25)
percentil_75 = np.percentile(amount,75)

fig, ax = plt.subplots()

ax.hist(dataset, bins=15, edgecolor="white")
ax.axvline(media, color="red", ls="-.", label=f"Media: {round(media,2)} €")
ax.axvline(mediana, color="orange", ls="--", label=f"Mediana: {round(mediana,2)} €")
ax.axvline(percentil_25, color="black", ls=":", label=f"Percentil 25 %: {round(percentil_25,2)} €")
ax.axvline(percentil_75, color="black", ls=":", label=f"Percentil 75 %: {round(percentil_75,2)} €")
ax.set_title('Rango de ingresos')
ax.set_xlabel('Valor compra')
ax.set_ylabel('N.º transacciones')
plt.legend(loc='upper right')
plt.tight_layout()

plt.show()

Ejercicio 2

Gráfico con dos variables numéricas

In [ ]:
# El codi següent per crear un marc de dades i suprimir files duplicades sempre s'executa i funciona com a preàmbul de l'script: 

# dataset = pandas.DataFrame(amount, age)
# dataset = dataset.drop_duplicates()

# Enganxeu o escriviu el codi de l'script aquí:

df_transaction_user = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid")
sns.scatterplot(y=dataset['amount'], x=dataset['age'])
plt.xlabel('Edad')
plt.ylabel('Valor compra (€)')
plt.title('Importe transacciones por edad')
plt.yticks(range(50,500,50))
plt.tight_layout()

plt.show()

### Ejercicio 3

##### Gráfico con una variable categórica

In [ ]:
# El codi següent per crear un marc de dades i suprimir files duplicades sempre s'executa i funciona com a preàmbul de l'script: 

# dataset = pandas.DataFrame(company_id)
# dataset = dataset.drop_duplicates()

# Enganxeu o escriviu el codi de l'script aquí:

df_company = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(data=dataset, y='country', order = dataset['country'].value_counts().index)

plt.title('N.º empresas por país')
plt.xlabel('Cantidad')
plt.ylabel('País')
plt.tight_layout()

plt.show()

### Ejercicio 4

##### Gráfico con una variable categórica y una numérica

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, period, declined)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

transacciones_mensuales = dataset['amount'].groupby([dataset['period']]).sum()

#Graficamos la evolución de los importes totales de las transacciones por periodo:
media_transacciones = transacciones_mensuales.mean()

ax = transacciones_mensuales.plot(kind='bar')
ax.axhline(y=media_transacciones, color='black', linestyle='--', label=f'Media trans.:{round(media_transacciones,2)} €')
ax.set_title('Importe transacciones por mes')
ax.set_ylabel('Importe (€)')
ax.set_xlabel('Periodos')
ax.legend(loc='upper right')
plt.tight_layout()

plt.show()

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, period, declined)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ventas_mensuales = dataset[dataset['declined']==0]['amount'].groupby([dataset['period']]).sum()

#Graficamos la evolución de los importes totales de las transacciones por periodo:
media_transacciones = ventas_mensuales.mean()

ax = ventas_mensuales.plot(kind='bar')
ax.axhline(y=media_transacciones, color='black', linestyle='--', label=f'Media trans.:{round(media_transacciones,2)} €')
ax.set_title('Importe ventas por mes')
ax.set_ylabel('Importe (€)')
ax.set_xlabel('Periodos')
ax.legend(loc='upper right')
plt.tight_layout()

plt.show()

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, period, declined)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Recuperamos las dos series de pandas con la suma de transacciones y ventas:
transacciones_mensuales = dataset['amount'].groupby([dataset['period']]).sum()
ventas_mensuales = dataset[dataset['declined']==0]['amount'].groupby([dataset['period']]).sum()

#Graficamos la evolución de los importes totales de las transacciones por periodo:
media_transacciones = transacciones_mensuales.mean()
media_ventas = ventas_mensuales.mean()

# Datos
barWidth = 0.4
bars1 = transacciones_mensuales.values
bars2 = ventas_mensuales.values


# Posición de las barras
r = np.arange(len(bars1))
r2 = r + barWidth

# Gráfica
fig, ax = plt.subplots(dpi=150)
ax.bar(r, bars1, color='#7f6d5f', width=barWidth, edgecolor='white', label='Operaciones')
ax.bar(r2, bars2, color='#557f2d', width=barWidth, edgecolor='white', label='Ventas')

# Xticks
ax.set_xlabel('Periodo', fontweight='bold')
ax.set_ylabel('Importe (€)')
ax.set_xticks(r + barWidth -0.5)
ax.set_xticklabels(transacciones_mensuales.index)
ax.tick_params(axis='x', labelrotation=45, labelsize='small')

# Medias
ax.axhline(y=media_transacciones, color='black', linestyle='--', label=f'Media trans.:{round(media_transacciones,2)} €')
ax.axhline(y=media_ventas, color='red', linestyle=':', label=f'Media ingresos: {round(media_ventas,2)} €')

# Legend
ax.legend(ncol=1, loc='upper right')
plt.tight_layout()

plt.show()

### Ejercicio 5

##### Gráfico con dos variables categóricas

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(undefined, undefined.1, undefined.2, undefined.3, undefined.4)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_declined = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Tenemos un problema con el orden del eje x. Vamos a ordenar el DataFrame antes de graficar:
dataset.sort_values('period', inplace=True)

sns.lineplot(x=dataset['period'], y=dataset['size'], marker='o', linestyle='-', sort=False)

plt.title('Transacciones declinadas por periodo')
plt.ylabel('Transacciones declinadas')
plt.xlabel('Periodo')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

### Ejercicio 6

##### Gráfico con tres variables

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(undefined, undefined.1, undefined.2)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_country_top = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Graficamos un boxplot para analizar las estadísticas por país y tipo de operación. Por encima, marcamos las operaciones por país para tener una referencia visual de la cantidad de operaciones de cada país:
custom_palette = {0: "green", 1: "red"}
sns.boxplot(y=dataset['country'], x=dataset['amount'], hue=dataset['declined'], palette=custom_palette)

# Añadimos un stripplot:
sns.stripplot(y='country', x='amount', hue='declined', data=dataset, palette="dark:orange", jitter=0.2, size=2.5)

plt.title("Estadísticas de venta por países", loc="center")
plt.ylabel('Países', fontweight='bold')
plt.xlabel('Importe (€)', fontweight='bold')

plt.legend(bbox_to_anchor=(1.05, 1), ncol=1, loc='upper left', title='Ventas')
plt.tight_layout()

plt.show()

### Ejercicio 7

##### Graficar un pairplot

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(undefined, undefined.1, undefined.2)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_country_top = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Graficamos un boxplot para analizar las estadísticas por país y tipo de operación. Por encima, marcamos las operaciones por país para tener una referencia visual de la cantidad de operaciones de cada país:
custom_palette = {0: "green", 1: "red"}
sns.boxplot(y=dataset['country'], x=dataset['amount'], hue=dataset['declined'], palette=custom_palette)

# Añadimos un stripplot:
# sns.stripplot(y='country', x='amount', hue='declined', data=dataset, palette="dark:orange", jitter=0.2, size=2.5)

plt.title("Estadísticas de venta por países", loc="center")
plt.ylabel('Países', fontweight='bold')
plt.xlabel('Importe (€)', fontweight='bold')

plt.legend(bbox_to_anchor=(1.05, 1), ncol=1, loc='upper left', title='Ventas')

plt.show()

## Nivel 2



### Ejercicio 1

Correlación de todas las variables numéricas

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(undefined, undefined.1, undefined.2, undefined.3, undefined.4)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_user_pr_wh_subset = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

g = sns.heatmap(dataset[['amount','age','prods_x_trans','price', 'weight']].corr(), annot=True)
g.set_xticklabels(['Importe', 'Edad', 'N.º prods.', 'Precio', 'Peso'])
g.set_yticklabels(['Importe', 'Edad', 'N.º prods.', 'Precio', 'Peso'])
plt.tight_layout()

plt.show()

### Ejercicio 2

##### Implementa un jointplot

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(undefined, undefined.1)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_user_pr_wh = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.jointplot(data=dataset, x='amount', y='price', kind='reg')
plt.xlabel('Importe transacciones')
plt.ylabel('Precio envío')
plt.tight_layout()


plt.show()

## Nivel 3

### Ejercicio 1

##### Implementa un violinplot combinado con otro tipo de gráfica.

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, country)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_country_top = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Vamos a añadir un swarmplot por encima para situar los valores de las transacciones

# Creamos la figura:
sns.set_context('notebook', font_scale=1.2)
fig, ax = plt.subplots(figsize=(8,5))

# Dibujamos el violin
ax = sns.violinplot(y="country", 
                  x="amount", 
                  data=dataset,
                  density_norm='count',
                  inner=None
              )
              

# Situamos por encima el swarmplot 
ax = sns.swarmplot(y="country",
                   x="amount",
                   data=dataset,
                   color="white", 
                   edgecolor="auto",
                   s=4, # Circle size
             )


ax.set_xlabel('Cantidad (€)')
ax.set_ylabel('Países')


ax.grid(axis='y')
ax.set_axisbelow(True)

plt.title('Distribución de importes por país')
plt.tight_layout()

plt.show()

### Ejercicio 2

##### Genera un FacetGrid para visualizar múltiples aspectos de los datos simultáneamente.

In [ ]:
# El código siguiente, que crea un dataframe y quita las filas duplicadas, siempre se ejecuta y actúa como un preámbulo del script: 

# dataset = pandas.DataFrame(amount, declined, prods_x_trans)
# dataset = dataset.drop_duplicates()

# Pegue o escriba aquí el código de script:

df_transaction_user_pr_wh = dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

g = sns.FacetGrid(dataset, col='declined', row='prods_x_trans', margin_titles=True)
g.map_dataframe(sns.histplot, x='amount', binwidth=20)
g.fig.suptitle("Distribución de transacciones en función del n.º de productos", fontsize=12, fontweight='bold')
g.fig.subplots_adjust(top=.93)
g.set_axis_labels('Importe (€)', 'N.º transacciones')
#g.set_titles(fontsize='small')

plt.show()